In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize, fsolve
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex
from scipy.optimize import least_squares

# sympy for regularizers

- start with two player

In [2]:
a = sp.symbols('a', positive=True, real=True)
lambda_ = sp.symbols('lambda')

In [3]:
c = sp.symbols('c1 c2', positive=True, real=True)
x = sp.symbols('x1 x2', positive=True, real=True)

In [4]:
L = sum([c[i] * x[i]**(1 + 1/a) for i in range(2)]) - lambda_ * (sum(x[i] for i in range(2)) - 1)
L

c1*x1**(1 + 1/a) + c2*x2**(1 + 1/a) - lambda*(x1 + x2 - 1)

In [5]:
derivatives = [sp.diff(L, x[i]) for i in range(2)] + [sp.diff(L, lambda_)]
derivatives[1]

c2*x2**(1 + 1/a)*(1 + 1/a)/x2 - lambda

In [6]:
equations = [sp.Eq(derivatives[i], 0) for i in range(3)]
equations[1]

Eq(c2*x2**(1 + 1/a)*(1 + 1/a)/x2 - lambda, 0)

In [7]:
solutions = sp.solve(equations, x + (lambda_,), dict=True)
solutions

[{lambda: c1*c2*(a + 1)/(a*(c1**a + c2**a)**(1/a)),
  x1: c2**a/(c1**a + c2**a),
  x2: c1**a/(c1**a + c2**a)}]

In [8]:
sp.simplify(c[0]**(-a) / (c[0]**(-a)+c[1]**(-a)))

c2**a/(c1**a + c2**a)

# a x log c + x log x

In [10]:
a = sp.symbols('a', positive=True, real=True)
lambda_ = sp.symbols('lambda')

In [11]:
c = sp.symbols('c1 c2', positive=True, real=True)
x = sp.symbols('x1 x2', positive=True, real=True)

In [12]:
L = sum([a*x[i] * sp.log(c[i]) + x[i] * sp.log(x[i]) for i in range(2)]) - lambda_ * (sum(x[i] for i in range(2)) - 1)
L

a*x1*log(c1) + a*x2*log(c2) - lambda*(x1 + x2 - 1) + x1*log(x1) + x2*log(x2)

In [13]:
derivatives = [sp.diff(L, x[i]) for i in range(2)] + [sp.diff(L, lambda_)]
derivatives[1]

a*log(c2) - lambda + log(x2) + 1

In [14]:
equations = [sp.Eq(derivatives[i], 0) for i in range(3)]
equations[1]

Eq(a*log(c2) - lambda + log(x2) + 1, 0)

In [15]:
solutions = sp.solve(equations, x + (lambda_,), dict=True)
solutions

[{lambda: log(c1**a*c2**a/(c1**a + c2**a)) + 1,
  x1: c2**a/(c1**a + c2**a),
  x2: c1**a/(c1**a + c2**a)}]

# x log c + a^-1 xlogx

In [22]:
a = sp.symbols('a', positive=True, real=True)
lambda_ = sp.symbols('lambda')

In [23]:
c = sp.symbols('c1 c2', positive=True, real=True)
x = sp.symbols('x1 x2', positive=True, real=True)

In [25]:
L = sum([x[i] * sp.log(c[i]) + a**(-1) * x[i] * sp.log(x[i]) for i in range(2)]) - lambda_ * (sum(x[i] for i in range(2)) - 1)
L

-lambda*(x1 + x2 - 1) + x1*log(c1) + x2*log(c2) + x1*log(x1)/a + x2*log(x2)/a

In [26]:
derivatives = [sp.diff(L, x[i]) for i in range(2)] + [sp.diff(L, lambda_)]
derivatives[1]

-lambda + log(c2) + log(x2)/a + 1/a

In [27]:
equations = [sp.Eq(derivatives[i], 0) for i in range(3)]
equations[1]

Eq(-lambda + log(c2) + log(x2)/a + 1/a, 0)

In [28]:
solutions = sp.solve(equations, x + (lambda_,), dict=True)
solutions

[{lambda: (log(c1**a*c2**a/(c1**a + c2**a)) + 1)/a,
  x1: c2**a/(c1**a + c2**a),
  x2: c1**a/(c1**a + c2**a)}]

#  log xc + a^-1 xlogx

In [29]:
a = sp.symbols('a', positive=True, real=True)
lambda_ = sp.symbols('lambda')

In [30]:
c = sp.symbols('c1 c2', positive=True, real=True)
x = sp.symbols('x1 x2', positive=True, real=True)

In [31]:
L = sum([ sp.log(x[i]*c[i]) + a**(-1) * x[i] * sp.log(x[i]) for i in range(2)]) - lambda_ * (sum(x[i] for i in range(2)) - 1)
L

-lambda*(x1 + x2 - 1) + log(c1*x1) + log(c2*x2) + x1*log(x1)/a + x2*log(x2)/a

In [32]:
derivatives = [sp.diff(L, x[i]) for i in range(2)] + [sp.diff(L, lambda_)]
derivatives[1]

-lambda + 1/x2 + log(x2)/a + 1/a

In [33]:
equations = [sp.Eq(derivatives[i], 0) for i in range(3)]
equations[1]

Eq(-lambda + 1/x2 + log(x2)/a + 1/a, 0)

In [34]:
solutions = sp.solve(equations, x + (lambda_,), dict=True)
solutions

NotImplementedError: could not solve a + x2*(log(x2) + 1) - x2*(-a + x2*log(1 - x2) + x2 - log(1 - x2) - 1)/(x2 - 1)